In [ ]:
import os
import json
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import os
import time
import random
import pandas as pd
import signal
import sys
import ctypes
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from markdownify import markdownify as md


# === [1] Konfigürasyon ===
CHROME_DRIVER_PATH = r"C:\Users\emirhan.gul\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
INPUT_FILE = "yeni_ilan_linkleri.csv"
OUTPUT_FILE = "ilan_aciklamalari.csv"
SAVE_INTERVAL = 20  # Her 20 ilanda bir kaydet
DRIVER_RESTART_INTERVAL = 150  # Her 150 ilanda bir driver'ı yeniden başlat
MAX_RETRIES = 3  # Maksimum deneme sayısı

# === [2] Global Değişkenler ===
driver = None
results_list = []
processed_urls = set()

# === [3] Chrome Driver Ayarları ===
def get_chrome_options():
    """Chrome seçeneklerini ayarla"""
    options = Options()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    return options

def wait_random(min_sec=1.0, max_sec=1.3):
    time.sleep(random.uniform(min_sec, max_sec))

def scroll_random():
    y = random.randint(300, 1200)
    driver.execute_script(f"window.scrollBy(0, {y});")

def extract_text_safe(by, selector):
    try:
        return driver.find_element(by, selector).text.strip()
    except:
        return None

def get_property_value(key):
    try:
        rows = driver.find_elements(By.CSS_SELECTOR, ".property-item")
        for row in rows:
            if key in row.text:
                return row.find_element(By.CLASS_NAME, "property-value").text.strip()
    except:
        return None
def get_with_timeout(driver, url, timeout=2):
    try:
        driver.set_page_load_timeout(timeout)
        driver.get(url)
    except TimeoutException:
        print(f"⚠️ Sayfa zaman aşımı, stop() uygulandı: {url}")
        driver.execute_script("window.stop()")  # manuel durdurma

def extract_parca_boya():
    try:
        categories = {
            "Orjinal": [],
            "Lokal boyalı": [],
            "Boyalı": [],
            "Değişmiş": [],
            "Belirtilmemiş": []
        }

        section = driver.find_element(By.CLASS_NAME, "car-damage-info")
        items = section.find_elements(By.CLASS_NAME, "car-damage-info-item")

        for item in items:
            try:
                label = item.find_element(By.TAG_NAME, "p").text.strip()
                ul = item.find_element(By.TAG_NAME, "ul")
                for li in ul.find_elements(By.TAG_NAME, "li"):
                    text = li.text.strip()
                    if text:
                        categories.get(label, []).append(text)
            except:
                continue

        return {k.lower().replace(" ", "_"): ", ".join(v) for k, v in categories.items()}

    except Exception as e:
        print(f"❌ Boya bilgisi alınamadı: {e}")
        return {k.lower().replace(" ", "_"): None for k in [
            "Orjinal", "Lokal boyalı", "Boyalı", "Değişmiş", "Belirtilmemiş"
        ]}

# === [C] Daha Önce Kaydedilen İlanları Yükle ===
if os.path.exists("ilan_tum_bilgiler.csv"):
    df_existing = pd.read_csv("ilan_tum_bilgiler.csv")
    done_urls = set(df_existing["url"].tolist())
else:
    df_existing = pd.DataFrame()
    done_urls = set()

results = []

